In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append("../")
import utils as ut

In [2]:
cardDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"
outdir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/graph_data/single_cell_expression/"

keys = ['ND', 'HFD8', 'HFD14']

for i, key in enumerate(keys):
    print(f"{key}...")
    labelPath = f"{cardDir}{key}_macrophage_clusters.csv"
    lf = pd.read_csv(labelPath)
    lf = lf.rename(columns={'sampleInfo':'key', 'cellId':'barcode'})
    lf['cellId'] = [f"{i+1}_{x+1}" for x in lf.index]

    # save the label file as a parquet file
    labelFile = f"{outdir}{key}_labels.pq"
    lf.to_parquet(labelFile, index=False)
    print(f'Labels done.')

    # get a label map for the expression data
    labelMap = pd.Series(lf['cellId'].values, index=lf['barcode']).to_dict()
        
    rnaPath = f"{cardDir}{key}_macrophage_rna.csv"
    rf = pd.read_csv(rnaPath)
    rf = rf.T
    new_header = rf.iloc[0] 
    rf = rf[1:] 
    rf.columns = new_header 

    # remap the index 
    rf['index'] = rf.index.to_series().map(labelMap)
    rf = rf.reset_index(drop=True)
    rf = rf.set_index('index')
    print(f"{rf.shape=}")

    # drop genes with no reads
    rf = rf.loc[:, rf.sum(axis=0) != 0]
    print(f"{rf.shape=}")

    # save the raw reads
    rawFile = f"{outdir}{key}_reads.pq"
    rf.to_parquet(rawFile)
    print(f'Reads done.')

    # normalize
    rf = ut.normalize(rf, 1e6)
    normFile = f"{outdir}{key}_cpm.pq"
    rf.to_parquet(normFile)
    print(f'Normed done.')


print('done')

ND...
Labels done.
rf.shape=(1261, 31053)
rf.shape=(1261, 17050)
Reads done.
Normed done.
HFD8...
Labels done.
rf.shape=(6123, 31053)
rf.shape=(6123, 18151)
Reads done.
Normed done.
HFD14...
Labels done.
rf.shape=(6436, 31053)
rf.shape=(6436, 17452)
Reads done.
Normed done.
done


In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
rf.head()

In [ ]:
sums = rf.sum(axis=0)
print(f"{sums.shape=}")

In [ ]:
rf.index.isin(labelMap.keys())

In [ ]:
rf.index[:10]

In [ ]:
labelMap['AAACGAACAGCAGTAG.1']

In [ ]:
rf.head()

In [ ]:
labelMap['AAACGAACAGCAGTAG.1']

In [ ]:
rf = rf.T
new_header = rf.iloc[0] 
rf = rf[1:] 
rf.columns = new_header 
rf.index.names = ['cellId']